In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [3]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv"
).drop(columns=["tweet_id"])
df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [6]:
# data preprocessing


# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)


def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)


def removing_numbers(text):
    """Remove numbers from the text."""
    text = "".join([char for char in text if not char.isdigit()])
    return text


def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)


def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub("[%s]" % re.escape(string.punctuation), " ", text)
    text = text.replace("؛", "")
    text = re.sub("\s+", " ", text).strip()
    return text


def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r"https?://\S+|www\.\S+")
    return url_pattern.sub(r"", text)


def normalize_text(df):
    """Normalize the text data."""
    try:
        df["content"] = df["content"].apply(lower_case)
        df["content"] = df["content"].apply(remove_stop_words)
        df["content"] = df["content"].apply(removing_numbers)
        df["content"] = df["content"].apply(removing_punctuations)
        df["content"] = df["content"].apply(removing_urls)
        df["content"] = df["content"].apply(lemmatization)
        return df
    except Exception as e:
        print(f"Error during text normalization: {e}")
        raise

<>:37: SyntaxWarning: invalid escape sequence '\s'
<>:37: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Usman\AppData\Local\Temp\ipykernel_4176\15599248.py:37: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub("\s+", " ", text).strip()


In [10]:
import nltk

nltk.download("wordnet")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Usman\AppData\Roaming\nltk_data...


True

In [11]:
df = normalize_text(df)
df.head()

,sentiment,content
0,empty,tiffanylue know listenin bad habit earlier sta...
1,sadness,layin n bed headache ughhhh waitin call
2,sadness,funeral ceremony gloomy friday
3,enthusiasm,want hang friend soon
4,neutral,dannycastillo want trade someone houston ticke...


In [12]:
df["sentiment"].value_counts()

sentiment
neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: count, dtype: int64

In [13]:
x = df["sentiment"].isin(["happiness", "sadness"])
df = df[x]

In [14]:
df["sentiment"].value_counts()

sentiment
happiness    5209
sadness      5165
Name: count, dtype: int64

In [15]:
df["sentiment"] = df["sentiment"].replace({"sadness": 0, "happiness": 1})
df.head()

C:\Users\Usman\AppData\Local\Temp\ipykernel_4176\4042218858.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["sentiment"] = df["sentiment"].replace({"sadness": 0, "happiness": 1})
C:\Users\Usman\AppData\Local\Temp\ipykernel_4176\4042218858.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["sentiment"] = df["sentiment"].replace({"sadness": 0, "happiness": 1})


,sentiment,content
1,0,layin n bed headache ughhhh waitin call
2,0,funeral ceremony gloomy friday
6,0,sleep im thinking old friend want married damn...
8,0,charviray charlene love miss
9,0,kelcouch sorry least friday


In [16]:
vectorizer = CountVectorizer(max_features=1000)
X = vectorizer.fit_transform(df["content"])
y = df["sentiment"]

In [18]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [19]:
import dagshub

mlflow.set_tracking_uri("https://dagshub.com/usmanuuu52/mlops-project.mlflow")
dagshub.init(repo_owner="usmanuuu52", repo_name="mlops-project", mlflow=True)

mlflow.set_experiment("Logistic Regression Baseline")

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\Usman\Desktop\mlops-project\mlops-project\myenv\Lib\site-packages\rich\live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=2525f4c1-bdc7-48aa-95c2-3094b407e93a&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=6f222a0f0dad60a5d7b9b7c018e22a4420d60bc53457a8dd54a1cce330909140




Accessing as usmanuuu52

Initialized MLflow to track repo "usmanuuu52/mlops-project"

Repository usmanuuu52/mlops-project initialized!

2024/11/20 14:41:32 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/3625214a363348f282a8c5ede37d8f87', creation_time=1732095692033, experiment_id='0', last_update_time=1732095692033, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [20]:
with mlflow.start_run():

    # Log preprocessing parameters
    mlflow.log_param("vectorizer", "Bag of Words")
    mlflow.log_param("num_features", 1000)
    mlflow.log_param("test_size", 0.2)

    # Model building and training
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # Log model parameters
    mlflow.log_param("model", "Logistic Regression")

    # Model evaluation
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Log evaluation metrics
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)

    # Log model
    mlflow.sklearn.log_model(model, "model")

    # Save and log the notebook
    import os

    notebook_path = "exp1_baseline_model.ipynb"
    os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
    mlflow.log_artifact(notebook_path)

    # Print the results for verification
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")

2024/11/20 14:42:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Accuracy: 0.7725301204819277
Precision: 0.7638483965014577
Recall: 0.774384236453202
F1 Score: 0.7690802348336595
🏃 View run efficient-gnu-898 at: https://dagshub.com/usmanuuu52/mlops-project.mlflow/#/experiments/0/runs/f29ac1760f42424180bc12d2af319261
🧪 View experiment at: https://dagshub.com/usmanuuu52/mlops-project.mlflow/#/experiments/0
